# Milestone 1: Exploratory Data Analysis and Data Preparation

## Food101 Dataset Analysis

This notebook performs comprehensive exploratory data analysis, data cleaning, and feature engineering on the Food101 dataset from torchvision.


## 1. Import Required Libraries


In [7]:
import torch
import torchvision
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)


PyTorch version: 2.5.1
Torchvision version: 0.20.1


## 2. Load the Food101 Dataset

Food101 is a dataset containing 101 food categories with 1000 images each. The dataset is split into training and test sets.


In [8]:
# Load the dataset (download if necessary)
train_dataset = datasets.Food101(root='./data', split='train', download=True)
test_dataset = datasets.Food101(root='./data', split='test', download=True)

print("Training dataset size:", len(train_dataset))
print("Test dataset size:", len(test_dataset))
print("Total images:", len(train_dataset) + len(test_dataset))


100%|██████████| 5.00G/5.00G [10:19<00:00, 8.07MB/s] 


Extracting ./data/food-101.tar.gz to ./data
Training dataset size: 75750
Test dataset size: 25250
Total images: 101000


## 3. Exploratory Data Analysis (EDA)

### 3.1 Dataset Structure and Basic Information


In [ ]:
# Get class names
class_names = train_dataset.classes
num_classes = len(class_names)

print(f"Number of classes: {num_classes}")
print(f"\nFirst 10 classes:")
for i in range(10):
    print(f"{i}: {class_names[i]}")


In [ ]:
# Display all food categories
print("All Food Categories:")
print("="*50)
for i, class_name in enumerate(class_names):
    print(f"{i+1:3d}. {class_name}")
    if (i+1) % 20 == 0:
        print()


### 3.2 Sample Images Visualization


In [ ]:
# Display sample images from different classes
fig, axes = plt.subplots(3, 4, figsize=(15, 10))
fig.suptitle('Sample Images from Different Food Categories', fontsize=16)

# Select 12 random classes
random_indices = np.random.choice(len(class_names), 12, replace=False)

for idx, ax in enumerate(axes.flat):
    class_idx = random_indices[idx]
    # Find first image of this class
    for i in range(len(train_dataset)):
        img, label = train_dataset[i]
        if label == class_idx:
            ax.imshow(img)
            ax.set_title(class_names[label])
            ax.axis('off')
            break

plt.tight_layout()
plt.show()


### 3.3 Image Properties Analysis

Analyzing image dimensions, sizes, and pixel value distributions.


In [ ]:
# Analyze image properties from a sample of images
sample_size = 1000
widths = []
heights = []
aspect_ratios = []

print(f"Analyzing {sample_size} images...")

for i in range(sample_size):
    img, _ = train_dataset[i]
    width, height = img.size
    widths.append(width)
    heights.append(height)
    aspect_ratios.append(width / height)

print("Image Dimensions Statistics:")
print(f"Width - Min: {min(widths)}, Max: {max(widths)}, Mean: {np.mean(widths):.2f}")
print(f"Height - Min: {min(heights)}, Max: {max(heights)}, Mean: {np.mean(heights):.2f}")
print(f"Aspect Ratio - Min: {min(aspect_ratios):.2f}, Max: {max(aspect_ratios):.2f}, Mean: {np.mean(aspect_ratios):.2f}")


In [ ]:
# Visualize image dimensions distribution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Width distribution
axes[0, 0].hist(widths, bins=50, color='blue', alpha=0.7)
axes[0, 0].set_title('Distribution of Image Widths')
axes[0, 0].set_xlabel('Width (pixels)')
axes[0, 0].set_ylabel('Frequency')

# Height distribution
axes[0, 1].hist(heights, bins=50, color='green', alpha=0.7)
axes[0, 1].set_title('Distribution of Image Heights')
axes[0, 1].set_xlabel('Height (pixels)')
axes[0, 1].set_ylabel('Frequency')

# Aspect ratio distribution
axes[1, 0].hist(aspect_ratios, bins=50, color='red', alpha=0.7)
axes[1, 0].set_title('Distribution of Aspect Ratios')
axes[1, 0].set_xlabel('Aspect Ratio (Width/Height)')
axes[1, 0].set_ylabel('Frequency')

# Width vs Height scatter
axes[1, 1].scatter(widths, heights, alpha=0.5, s=10)
axes[1, 1].set_title('Width vs Height')
axes[1, 1].set_xlabel('Width (pixels)')
axes[1, 1].set_ylabel('Height (pixels)')

plt.tight_layout()
plt.show()


### 3.4 Class Distribution Analysis


In [ ]:
# Count samples per class
train_labels = [train_dataset[i][1] for i in range(len(train_dataset))]
test_labels = [test_dataset[i][1] for i in range(len(test_dataset))]

train_counts = pd.Series(train_labels).value_counts().sort_index()
test_counts = pd.Series(test_labels).value_counts().sort_index()

print("Class Distribution Statistics:")
print(f"Training samples per class - Min: {train_counts.min()}, Max: {train_counts.max()}, Mean: {train_counts.mean():.2f}")
print(f"Test samples per class - Min: {test_counts.min()}, Max: {test_counts.max()}, Mean: {test_counts.mean():.2f}")


In [ ]:
# Visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

axes[0].bar(range(len(train_counts)), train_counts.values)
axes[0].set_title('Training Set - Samples per Class')
axes[0].set_xlabel('Class Index')
axes[0].set_ylabel('Number of Samples')
axes[0].grid(axis='y', alpha=0.3)

axes[1].bar(range(len(test_counts)), test_counts.values)
axes[1].set_title('Test Set - Samples per Class')
axes[1].set_xlabel('Class Index')
axes[1].set_ylabel('Number of Samples')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()


### 3.5 Pixel Value Distribution Analysis

Analyzing the distribution of pixel values across RGB channels.


In [ ]:
# Analyze pixel values from sample images
sample_size = 100
r_values = []
g_values = []
b_values = []

print(f"Analyzing pixel values from {sample_size} images...")

for i in range(sample_size):
    img, _ = train_dataset[i]
    img_array = np.array(img)
    
    r_values.extend(img_array[:, :, 0].flatten())
    g_values.extend(img_array[:, :, 1].flatten())
    b_values.extend(img_array[:, :, 2].flatten())

r_values = np.array(r_values)
g_values = np.array(g_values)
b_values = np.array(b_values)

print("\nPixel Value Statistics (0-255 range):")
print(f"Red Channel - Mean: {r_values.mean():.2f}, Std: {r_values.std():.2f}")
print(f"Green Channel - Mean: {g_values.mean():.2f}, Std: {g_values.std():.2f}")
print(f"Blue Channel - Mean: {b_values.mean():.2f}, Std: {b_values.std():.2f}")


In [ ]:
# Visualize pixel value distributions
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].hist(r_values, bins=50, color='red', alpha=0.7)
axes[0].set_title('Red Channel Distribution')
axes[0].set_xlabel('Pixel Value')
axes[0].set_ylabel('Frequency')

axes[1].hist(g_values, bins=50, color='green', alpha=0.7)
axes[1].set_title('Green Channel Distribution')
axes[1].set_xlabel('Pixel Value')
axes[1].set_ylabel('Frequency')

axes[2].hist(b_values, bins=50, color='blue', alpha=0.7)
axes[2].set_title('Blue Channel Distribution')
axes[2].set_xlabel('Pixel Value')
axes[2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()


## 4. Data Cleaning

### 4.1 Check for Missing or Corrupted Data


In [ ]:
# Check for corrupted images in training set
print("Checking for corrupted images in training set...")
corrupted_train = []

for i in range(len(train_dataset)):
    try:
        img, label = train_dataset[i]
        if img is None:
            corrupted_train.append(i)
        # Verify image can be converted to array
        np.array(img)
    except Exception as e:
        corrupted_train.append(i)
        print(f"Error at index {i}: {e}")

print(f"Number of corrupted images in training set: {len(corrupted_train)}")

# Check for corrupted images in test set
print("\nChecking for corrupted images in test set...")
corrupted_test = []

for i in range(len(test_dataset)):
    try:
        img, label = test_dataset[i]
        if img is None:
            corrupted_test.append(i)
        # Verify image can be converted to array
        np.array(img)
    except Exception as e:
        corrupted_test.append(i)
        print(f"Error at index {i}: {e}")

print(f"Number of corrupted images in test set: {len(corrupted_test)}")


### 4.2 Identify Outliers in Image Dimensions

Using statistical methods to detect images with unusual dimensions.


In [ ]:
# Detect outliers using IQR method
widths_array = np.array(widths)
heights_array = np.array(heights)

# Calculate quartiles for width
q1_width = np.percentile(widths_array, 25)
q3_width = np.percentile(widths_array, 75)
iqr_width = q3_width - q1_width
lower_bound_width = q1_width - 1.5 * iqr_width
upper_bound_width = q3_width + 1.5 * iqr_width

# Calculate quartiles for height
q1_height = np.percentile(heights_array, 25)
q3_height = np.percentile(heights_array, 75)
iqr_height = q3_height - q1_height
lower_bound_height = q1_height - 1.5 * iqr_height
upper_bound_height = q3_height + 1.5 * iqr_height

# Find outliers
width_outliers = np.sum((widths_array < lower_bound_width) | (widths_array > upper_bound_width))
height_outliers = np.sum((heights_array < lower_bound_height) | (heights_array > upper_bound_height))

print("Outlier Detection Results:")
print(f"Width outliers: {width_outliers} images")
print(f"  Lower bound: {lower_bound_width:.2f}, Upper bound: {upper_bound_width:.2f}")
print(f"Height outliers: {height_outliers} images")
print(f"  Lower bound: {lower_bound_height:.2f}, Upper bound: {upper_bound_height:.2f}")


In [ ]:
# Visualize outliers with box plots
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].boxplot(widths)
axes[0].set_title('Width Distribution - Box Plot')
axes[0].set_ylabel('Width (pixels)')
axes[0].grid(axis='y', alpha=0.3)

axes[1].boxplot(heights)
axes[1].set_title('Height Distribution - Box Plot')
axes[1].set_ylabel('Height (pixels)')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()


### 4.3 Data Cleaning Summary

**Findings:**
- The Food101 dataset appears to be well-maintained with no missing values
- Images have varying dimensions, which is expected for real-world food photography
- No corrupted images were found during our validation

**Cleaning Actions:**
- No missing data to handle as the dataset is complete
- Outliers in dimensions are legitimate variations in food photography and should be preserved
- All images can be successfully loaded and processed

**Rationale:**
- For image classification tasks, varying dimensions are normal and will be handled through preprocessing transformations
- We will address dimension standardization in the feature engineering phase
- The dataset does not require removal of any samples


## 5. Feature Engineering

### 5.1 Image Transformations and Preprocessing

Creating standardized preprocessing pipelines for model training.


In [ ]:
# Define transformation pipeline for training data
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define transformation pipeline for test data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("Training Transformations:")
print("1. Resize to 256x256")
print("2. Center crop to 224x224")
print("3. Random horizontal flip (data augmentation)")
print("4. Random rotation up to 10 degrees (data augmentation)")
print("5. Convert to tensor")
print("6. Normalize using ImageNet statistics")

print("\nTest Transformations:")
print("1. Resize to 256x256")
print("2. Center crop to 224x224")
print("3. Convert to tensor")
print("4. Normalize using ImageNet statistics")


In [ ]:
# Create datasets with transformations
train_dataset_transformed = datasets.Food101(root='./data', split='train', transform=train_transform)
test_dataset_transformed = datasets.Food101(root='./data', split='test', transform=test_transform)

print("Transformed datasets created successfully")
print(f"Training set size: {len(train_dataset_transformed)}")
print(f"Test set size: {len(test_dataset_transformed)}")


### 5.2 Visualize Transformation Effects


In [ ]:
# Compare original and transformed images
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle('Original vs Transformed Images', fontsize=16)

# Select 4 random images
random_indices = np.random.choice(len(train_dataset), 4, replace=False)

for idx, img_idx in enumerate(random_indices):
    # Original image
    orig_img, label = train_dataset[img_idx]
    axes[0, idx].imshow(orig_img)
    axes[0, idx].set_title(f'Original: {class_names[label]}')
    axes[0, idx].axis('off')
    
    # Transformed image
    trans_img, _ = train_dataset_transformed[img_idx]
    # Denormalize for visualization
    trans_img_display = trans_img.permute(1, 2, 0).numpy()
    trans_img_display = trans_img_display * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    trans_img_display = np.clip(trans_img_display, 0, 1)
    
    axes[1, idx].imshow(trans_img_display)
    axes[1, idx].set_title('Transformed')
    axes[1, idx].axis('off')

plt.tight_layout()
plt.show()


### 5.3 Create Metadata Features

Extracting additional features from images that can be useful for analysis.


In [ ]:
# Create metadata dataframe for a sample of images
sample_size = 1000
metadata = []

print(f"Creating metadata for {sample_size} images...")

for i in range(sample_size):
    img, label = train_dataset[i]
    img_array = np.array(img)
    
    # Extract features
    width, height = img.size
    aspect_ratio = width / height
    total_pixels = width * height
    
    # Color statistics
    mean_r = img_array[:, :, 0].mean()
    mean_g = img_array[:, :, 1].mean()
    mean_b = img_array[:, :, 2].mean()
    
    std_r = img_array[:, :, 0].std()
    std_g = img_array[:, :, 1].std()
    std_b = img_array[:, :, 2].std()
    
    # Brightness (average of RGB)
    brightness = (mean_r + mean_g + mean_b) / 3
    
    metadata.append({
        'image_index': i,
        'class_label': label,
        'class_name': class_names[label],
        'width': width,
        'height': height,
        'aspect_ratio': aspect_ratio,
        'total_pixels': total_pixels,
        'mean_red': mean_r,
        'mean_green': mean_g,
        'mean_blue': mean_b,
        'std_red': std_r,
        'std_green': std_g,
        'std_blue': std_b,
        'brightness': brightness
    })

metadata_df = pd.DataFrame(metadata)
print("Metadata created successfully")
print(f"\nMetadata shape: {metadata_df.shape}")
print("\nFirst few rows:")
print(metadata_df.head())


In [ ]:
# Statistical summary of metadata
print("Metadata Statistical Summary:")
print(metadata_df.describe())


### 5.4 Feature Correlation Analysis


In [ ]:
# Create correlation matrix for numerical features
numerical_features = ['width', 'height', 'aspect_ratio', 'total_pixels', 
                      'mean_red', 'mean_green', 'mean_blue',
                      'std_red', 'std_green', 'std_blue', 'brightness']

correlation_matrix = metadata_df[numerical_features].corr()

# Visualize correlation matrix
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()


### 5.5 Feature Engineering Summary

**Features Created:**

1. **Image Standardization (224x224):**
   - Rationale: Ensures consistent input size for neural networks
   - Method: Resize to 256x256, then center crop to 224x224

2. **Data Augmentation (Training Only):**
   - Random horizontal flip: Increases training diversity without changing food appearance
   - Random rotation (10 degrees): Simulates different camera angles
   - Rationale: Improves model generalization and reduces overfitting

3. **Normalization:**
   - Using ImageNet statistics (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
   - Rationale: Helps with faster convergence and better performance when using pretrained models

4. **Metadata Features:**
   - Aspect ratio: Captures image proportions
   - Total pixels: Represents image size
   - Color statistics (RGB means and std): Captures color characteristics
   - Brightness: Overall luminance of the image
   - Rationale: These features can be used for exploratory analysis and understanding dataset characteristics

**Categorical Encoding:**
- Class labels are already encoded as integers (0-100)
- No additional encoding needed for the classification task


## 6. Summary and Key Findings

### Dataset Overview
- **Dataset:** Food101 from torchvision
- **Total Images:** 101,000 (75,750 training, 25,250 test)
- **Classes:** 101 food categories
- **Distribution:** Balanced dataset with 750 training and 250 test images per class

### Key Findings from EDA

1. **Image Dimensions:**
   - Images have varying dimensions, typical of real-world photography
   - Width and height range varies significantly
   - Aspect ratios are diverse, reflecting different food presentation styles

2. **Class Distribution:**
   - Perfectly balanced dataset across all 101 classes
   - Training: 750 images per class
   - Test: 250 images per class

3. **Pixel Value Statistics:**
   - RGB channels show typical natural image distributions
   - Color statistics vary across food categories
   - Brightness levels are diverse

### Data Quality
- No missing or corrupted images found
- All images load successfully
- No data cleaning required
- Outliers in dimensions are legitimate variations

### Preprocessing Pipeline Ready
- Training data includes augmentation for better generalization
- Test data uses consistent preprocessing without augmentation
- Standardized 224x224 input size for model compatibility
- ImageNet normalization applied for transfer learning

### Next Steps
1. Model selection and architecture design
2. Training pipeline implementation
3. Model evaluation and performance analysis
4. Hyperparameter tuning if needed


## 7. Save Processed Information


In [ ]:
# Save metadata to CSV file
metadata_df.to_csv('food101_metadata.csv', index=False)
print("Metadata saved to food101_metadata.csv")

# Save class names for reference
with open('food101_classes.txt', 'w') as f:
    for i, class_name in enumerate(class_names):
        f.write(f"{i},{class_name}\n")
print("Class names saved to food101_classes.txt")

print("\nMilestone 1 Complete!")
print("Data exploration, cleaning, and feature engineering finished successfully.")
